# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers
from keras.layers import LSTM

import tensorflow as tf

Using TensorFlow backend.


In [2]:
print(tf.__version__)
# print(tf.__version__sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)))

1.9.0


# Define architectures

Define each one of the different architectures to be tested.

In [3]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.10

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_4(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

In [4]:
def RULmodel_SN_1(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_3(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

In [4]:
def get_compiled_model(model_def, shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [5]:
#Define the usable models for this notebook

models = {'shallow-20-20':RULmodel_SN_4}

#models = {'shallow-250-100':RULmodel_SN_4, 'shallow-100-50':RULmodel_SN_1, 'shallow-50-20':RULmodel_SN_2,
#          'shallow-20-20':RULmodel_SN_3, 'shallow-20':RULmodel_SN_5, 'shallow-10':RULmodel_SN_6}
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Process Data

In [6]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

# Build the model

In [7]:
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
shapeLSTM = (window_size,nFeatures)
model = get_compiled_model(models['shallow-20-20'], shapeSN, model_type='ann')

tModel = SequenceTunableModelRegression('ann20', model, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [8]:
#For LSTM
# tModel.data_handler.data_scaler = min_max_scaler
# tModel.data_scaler = None

#For ANN
tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]
 [-0

# Test on dataset 1

In [10]:
iterations = 10
tModel.epochs = 100
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 30
windowStride = 1
constRul = 140

file = open("results/MLP/ResultsDataset_1.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

for key, model_def in models.items():
  
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
  
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
      
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
      
        input_shape = windowSize*num_features #For simple ANN
      
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()

        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
          
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
          
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        print("Results for model " + key)
  
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
          
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        
file.close()

For model shallow-250-100
Computing for dataset 1
100/100 [==============================] - 0s 1ms/step
Results for model shallow-250-100
DescribeResult(nobs=10, minmax=(array([15.44312145]), array([16.48544813])), mean=array([15.8214856]), variance=array([0.13954832]), skewness=array([0.70844646]), kurtosis=array([-0.91240078]))
DescribeResult(nobs=10, minmax=(array([3.98989722]), array([6.4733304])), mean=array([5.19731842]), variance=array([0.77465176]), skewness=array([0.05257875]), kurtosis=array([-1.28455042]))
DescribeResult(nobs=10, minmax=(array([9.37768724]), array([12.04274834])), mean=array([10.29368036]), variance=array([0.76996355]), skewness=array([1.0663643]), kurtosis=array([-0.18259191]))
For model shallow-100-50
Computing for dataset 1
100/100 [==============================] - 0s 2ms/step
Results for model shallow-100-50
DescribeResult(nobs=10, minmax=(array([15.26138919]), array([17.61703721])), mean=array([15.96005971]), variance=array([0.58388217]), skewness=arr

# Test on all Datasets

In [11]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 150
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
window_sizes = {1:17,2:17,3:17,4:17}
strides = {1:1,2:1,3:1,4:1}
max_ruls = {1:129, 2:129, 3:129, 4:129}
num_features = len(selected_features)

input_shape = None
models = {'shallow-20-20':RULmodel_SN_4}

#For each model
for key, model_def in models.items():
    file = open("results/MLP/ResultsDatasets_singleSet"+key+".csv", "w")
    csvfile = csv.writer(file, lineterminator='\n')
    
    print("Generating statistics for model " + key)

    #For each dataset
    for i in range(1,5):
        
        print("Working on dataset " + str(i))
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = window_sizes[i]*num_features #For simple ANN
        #input_shape = (window_sizes[i],num_features) #For RNN
        
        print(input_shape)
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()
        
        #tModel.print_data()
        
        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time
            
        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
    
    file.close()

Generating statistics for model shallow-20-20
Working on dataset 1
238
100/100 [==============================] - 1s 10ms/step
Results for model shallow-20-20
DescribeResult(nobs=10, minmax=(array([16.74604431]), array([17.23861943])), mean=array([17.06463444]), variance=array([0.0325017]), skewness=array([-0.88353254]), kurtosis=array([-0.61935145]))
DescribeResult(nobs=10, minmax=(array([8.08271055]), array([9.26545247])), mean=array([8.64814575]), variance=array([0.20635946]), skewness=array([-0.04830206]), kurtosis=array([-1.50147762]))
DescribeResult(nobs=10, minmax=(array([17.28190351]), array([20.75944268])), mean=array([18.59064881]), variance=array([1.26079365]), skewness=array([0.53965178]), kurtosis=array([-0.70071753]))
Working on dataset 2
238
259/259 [==============================] - 1s 4ms/step
Results for model shallow-20-20
DescribeResult(nobs=10, minmax=(array([29.77055112]), array([30.12278605])), mean=array([29.9410427]), variance=array([0.01371166]), skewness=arra